## **Notebook to Clean and Transform applicable variables**

**Transformations to do:**

**These columns will be excluded from the dataset:** (pdf,href formula, href, Case Number, CaseNumber.1, original order)

**1. Year:** Convert to Integer, from '2025.0' to '2025'

**2. Date:** convert format to real date '27th November' to '11/27/2025'. Use the Column [Year] to concatenate the year into the normalized Date

**3. Type:** Normalize different iterations of the same value, example: 'Unprovoked' & 'unprovoked'

**4. Country:** Only 50 nulls, explore those records to see if there is opportunity to extrapolate it from other columns like 'Location' or 'State'

**5.  State:** N/A

**6. Location:** N/A

**7. Activity:** Normalize different iterations of the same value. Use NLP to clustter similar activities.

**8. Name:** N/A

**9. Sex:** Normalize different iterations of the same value

**10. Age:** Remove non numerical values, explore those records

**11. Injury:** Normalize different iterations of the same value and Explore creating categories (clusters?)

**12. Fatal Y/N:** Normalize different iterations of the same value, this must be a binary column, maybe create a different column for values that are not Y/N or delete those records

**13. Time:** Normalize to timestamp, create a new column called 'time_cluster' with 'Afternoon/Morning/Night'

**14. Species:** Normalize different iterations of the same value, Create a new column with the size when available called 'shark_size' that contains meters, ie. '3m'.

**15. Source:** N/A

### **Libraries**

In [13]:
# Libraries
import os, re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from datetime import datetime

### **Adjust Path**

In [21]:
# Get the current notebook's directory
NOTEBOOK_DIR = Path.cwd()

# Navigate to project root (assuming notebook is in 'notebooks' folder)
if NOTEBOOK_DIR.name == 'notebooks':
    PROJECT_ROOT = NOTEBOOK_DIR.parent
else:
    # If already in project root or elsewhere
    PROJECT_ROOT = Path(r'C:\Users\ynk615\OneDrive - Corteva\Documentos\dataprojects\sharks_p1_attacks')

# Define paths
RAW_PATH = PROJECT_ROOT / 'data' / 'raw' / 'GSAF5.csv'
OUT_DIR = PROJECT_ROOT / 'data' / 'processed'

# Create output directory
OUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"Working from: {NOTEBOOK_DIR}")
print(f"Project root: {PROJECT_ROOT}")
print(f"Raw data: {RAW_PATH}")
print(f"Output dir: {OUT_DIR}")

# Load Data
df = pd.read_csv(RAW_PATH)
print(f"\nData shape: {df.shape}")


Working from: c:\Users\ynk615\OneDrive - Corteva
Project root: C:\Users\ynk615\OneDrive - Corteva\Documentos\dataprojects\sharks_p1_attacks
Raw data: C:\Users\ynk615\OneDrive - Corteva\Documentos\dataprojects\sharks_p1_attacks\data\raw\GSAF5.csv
Output dir: C:\Users\ynk615\OneDrive - Corteva\Documentos\dataprojects\sharks_p1_attacks\data\processed

Data shape: (7058, 21)


After data transformations

- output_file = OUT_DIR / 'GSAF5_cleaned.csv'
- df_clean.to_csv(output_file, index=False)
- print(f"Saved cleaned data to: {output_file}")

### **Transformations & Cleanup**

#### T&C | New Dataframe

In [22]:
# Copy dataframe df to df_clean
df_clean = df.copy()

#### T&C | Exclude Columns

In [23]:
# Drop columns that are not needed for analysis
DROP_COLS = ["pdf", "href formula", "href","Case Number", "Case Number.1", "original order"]

df_clean = df_clean.drop(columns=[c for c in DROP_COLS if c in df.columns], errors="ignore")

df_clean.head() 

,Date,Year,Type,Country,State,Location,Activity,Name,Sex,Age,Injury,Fatal Y/N,Time,Species,Source
0,27th November,2025.0,Unprovoked,Australia,NSW,Crowdy Bay,Swimming,Lukas Schindler,M,26,Serious leg injuries,N,0630hrs,3m Bull shark,Media: Todd Smith: Andy Currie: Simon De March...
1,27th November,2025.0,Unprovoked,Australia,NSW,Crowdy Bay,Swimming,Livia Mulheim,F,25,Not stated,Y,0630hrs,3m Bull shark,Media: Todd Smith: Andy Currie: Simon De March...
2,10th November,2025.0,Unprovoked,Australia,Western Australia,Prevelly Beach Magaret River,Foil Boarding,Andy McDonald,M,61,No Injury to self,N,1745hrs,Great White Shark,Andy Currie
3,9th November,2025.0,Unprovoked,French Polynesia,Marquesas Islands,Hakahau Bay,Swimming,Not stated (Dentist),M,40,Deep Gash to bicep,N,Not stated,3m shark,Andrew Currie
4,5th November,2025.0,Unprovoked,USA,Hawaii,Pine Trees Hanalei Bay Kaui,Swimming,Chance Swanson,M,?,Injuries to legs,N,Mid afternoon,Unknown,James Kingsley: Andy Currie: Beat of Hawaii:


#### 1. T&C | Year

In [24]:
# Convert to Integer, from '2025.0' to '2025'
df_clean['Year'] = pd.to_numeric(df_clean['Year'], errors='coerce').astype('Int64')

df_clean.head(5)

,Date,Year,Type,Country,State,Location,Activity,Name,Sex,Age,Injury,Fatal Y/N,Time,Species,Source
0,27th November,2025,Unprovoked,Australia,NSW,Crowdy Bay,Swimming,Lukas Schindler,M,26,Serious leg injuries,N,0630hrs,3m Bull shark,Media: Todd Smith: Andy Currie: Simon De March...
1,27th November,2025,Unprovoked,Australia,NSW,Crowdy Bay,Swimming,Livia Mulheim,F,25,Not stated,Y,0630hrs,3m Bull shark,Media: Todd Smith: Andy Currie: Simon De March...
2,10th November,2025,Unprovoked,Australia,Western Australia,Prevelly Beach Magaret River,Foil Boarding,Andy McDonald,M,61,No Injury to self,N,1745hrs,Great White Shark,Andy Currie
3,9th November,2025,Unprovoked,French Polynesia,Marquesas Islands,Hakahau Bay,Swimming,Not stated (Dentist),M,40,Deep Gash to bicep,N,Not stated,3m shark,Andrew Currie
4,5th November,2025,Unprovoked,USA,Hawaii,Pine Trees Hanalei Bay Kaui,Swimming,Chance Swanson,M,?,Injuries to legs,N,Mid afternoon,Unknown,James Kingsley: Andy Currie: Beat of Hawaii:


#### 2. T&C | Date

Notes:

Messy dataset, lots of different formats and some entries with no dates at all (ie. 'No date', 'Early 1930s','World War II', etc)

In [26]:
def parse_date_flexible(row):
    """
    Try to parse various date formats. Return NaT if unable to parse.
    """
    date_str = str(row['Date']).strip()
    year = row['Year']
    
    # Handle completely non-date values
    non_dates = ['no date', 'before', 'after', 'circa', 'early', 'late', 'mid', 
                 'summer', 'winter', 'fall', 'spring', 'world war', 'ca .', 
                 'between', 'prior', 'letter', 'last incident', 'probably']
    
    if any(nd in date_str.lower() for nd in non_dates):
        return pd.NaT
    
    # Handle pure numbers (Excel date serial numbers)
    if date_str.isdigit():
        try:
            # Convert Excel serial date to datetime
            date_obj = pd.to_datetime('1899-12-30') + pd.Timedelta(days=int(date_str))
            return date_obj
        except:
            return pd.NaT
    
    # Remove "Reported" prefix
    date_str = pd.Series(date_str).str.replace(r'Reported\s+', '', regex=True, case=False).iloc[0]
    
    # Remove ordinal suffixes (st, nd, rd, th)
    date_clean = pd.Series(date_str).str.replace(r'(\d+)(st|nd|rd|th)', r'\1', regex=True).iloc[0]
    
    # Try different date formats
    formats_to_try = [
        ('%d %B', True),           # 10 November (needs year appended)
        ('%d %B %Y', False),       # 10 November 2025 (has year)
        ('%d-%b-%Y', False),       # 03-Dec-1882
        ('%d-%b-%y', True),        # 11-Jun-25 (needs year replacement)
        ('%b-%Y', False),          # Apr-1863
        ('%b-%d-%Y', False),       # Aug-24-1806
        ('%d %b-%Y', False),       # 02 Nov-2023
    ]
    
    # Try parsing with each format
    for fmt, needs_year_append in formats_to_try:
        try:
            if needs_year_append:
                # For formats like "%d %B" that don't include year
                date_obj = pd.to_datetime(f"{date_clean} {year}", format=f"{fmt} %Y")
            else:
                # For formats that already have year
                date_obj = pd.to_datetime(date_clean, format=fmt)
                # Replace with Year column value if available
                if year and not pd.isna(year):
                    date_obj = date_obj.replace(year=int(year))
            return date_obj
        except:
            continue
    
    # If all formats fail, return NaT
    return pd.NaT

# Apply the parsing function
df_clean['Date_Original'] = df_clean['Date']  # Keep original for reference
df_clean['Date_Parsed'] = df_clean.apply(parse_date_flexible, axis=1)

# Create a flag for successfully parsed dates
df_clean['Date_Valid'] = ~df_clean['Date_Parsed'].isna()

# Format valid dates as MM/DD/YYYY
df_clean['Date'] = df_clean['Date_Parsed'].dt.strftime('%m/%d/%Y')

# Check how many dates were successfully parsed
print(f"Successfully parsed: {df_clean['Date_Valid'].sum()} out of {len(df_clean)} records")
print(f"Failed to parse: {(~df_clean['Date_Valid']).sum()} records")
print(f"Success rate: {df_clean['Date_Valid'].sum() / len(df_clean) * 100:.1f}%")

# View records that failed to parse
if (~df_clean['Date_Valid']).sum() > 0:
    print("\nRecords that could not be parsed:")
    print(df_clean[~df_clean['Date_Valid']][['Date_Original', 'Year']].head(20))

Successfully parsed: 6421 out of 7058 records
Failed to parse: 637 records
Success rate: 91.0%

Records that could not be parsed:
              Date_Original  Year
178   Reported 14-June 2023  2023
220            29 Jan--2023  2023
221            08-Jan--2023  2023
224                  23-Jan  2023
326            11-Dec-2021`  2021
493              10-Jul-202  2020
713   Reported 09-Jul-2018.  2018
724                Jun-1018  2018
732                  18-May  2018
808                  17-Sep  2017
853              2017.06.05  2017
905             00.Feb.2017  2017
912    Reported 08-Jan-2017  <NA>
953                  16-Sep  2016
1008                 16-May  2016
1098                 15-Sep  2015
1231                 14-Sep  2014
1252                 14-Aug  2014
1446                 12-Dec  2012
1529                 13-Apr  2012
